In [1]:
import boto3
import os
from os import walk

In [2]:
AWS_ACCESS_KEY_ID = "your_access_key"
AWS_SECRET_ACCESS_KEY = "your_secret_key"
AWS_SESSION_TOKEN = "your_session_token"

In [3]:
def upload_s3_folder(bucket_name, s3_folder, local_dir=None):
    s3 = boto3.resource("s3")
    bucket = s3.Bucket(bucket_name)

    for path, _, files in os.walk(local_dir):
        directory_name = path.replace(local_dir, s3_folder)

        for file in files:
            bucket.upload_file(
                os.path.join(path, file), os.path.join(directory_name, file)
            )

In [4]:
bucket_ = "f14-datalake-landing-dev"

In [5]:
local_dir = "/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/"

f = []

for dirpath, dirnames, filenames in walk(local_dir):
    f.extend(filenames)
    break

full_filename = []

for i in f:
    full_filename.append(local_dir + i)

In [6]:
print(len(f))
print(f[:5])

294
['tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv', 'tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-03.csv', 'tribe_wod-na-us-detailsalestrafficbychilditem-2021-06-06.csv', 'tribe_wod-na-ca-detailsalestrafficbychilditem-2021-10-12.csv', 'tribe_wod-na-ca-detailsalestrafficbychilditem-2021-10-06.csv']


In [7]:
full_filename[:5]

['/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv',
 '/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-03.csv',
 '/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-06-06.csv',
 '/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-ca-detailsalestrafficbychilditem-2021-10-12.csv',
 '/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-ca-detailsalestrafficbychilditem-2021-10-06.csv']

In [11]:
# prefix creation
full_converted_files = []
for file in full_filename:
    file0 = file

    file = file.split("/")[7]
    # print(file)
    pieces = file.split("-")
    # print(pieces)

    y = pieces[4]
    m = pieces[5]
    d = pieces[6].split(".")[0]

    prefix0 = (
        "scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/"
    )
    prefix1 = f"p_creation_dt={y}-{m}-{d}/"

    brand = pieces[0].upper()
    prefix2 = f"p_brand_id={brand}/"

    prefix3 = f"p_region_id={pieces[1].upper()}/"
    prefix4 = f"p_country_id={pieces[2].upper()}/"
    prefix5 = "-".join(pieces[:4])
    prefix6 = f"-{y}-{m}-{d}.csv"
    # if pieces[2] == "us":
    #     prefix5 = f"{brand}-United-States-{y}-{m}-{d}.csv"
    # elif pieces[2] == "mx":
    #     prefix5 = f"{brand}-Mexico-{y}-{m}-{d}.csv"
    # else:
    #     prefix5 = f"{brand}-Canada-{y}-{m}-{d}.csv"

    final_prefix = prefix0 + prefix1 + prefix2 + prefix3 + prefix4 + prefix5 + prefix6
    # print(final_prefix)

    full_converted_files.append((file0, final_prefix))

print(full_converted_files[:1])

[('/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv', 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-08-17/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=US/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv')]


In [12]:
print(full_converted_files[:5])
print(len(full_converted_files))

[('/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv', 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-08-17/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=US/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv'), ('/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-03.csv', 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-08-03/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=US/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-03.csv'), ('/Users/emif/Downloads/files_work/seller_central/tribe_wod-detailsalestrafficbychilditem/tribe_wod-na-us-detailsalestrafficbychilditem-2021-06-06.csv', 'scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-06-06/p_

In [13]:
# Cliente S3
s3 = boto3.resource(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    aws_session_token=AWS_SESSION_TOKEN,
)

bucket = s3.Bucket(bucket_)

print(f"Files to be uploaded: {len(full_converted_files)}")
c = 0
for file0, file1 in full_converted_files:

    # print(file0)
    # print(file1)

    c += 1
    try:
        bucket.upload_file(file0, file1)
        print(f"Item Nbr:{c}:  {file1} successfully copied")
    except:
        print("ERROR")

Files to be uploaded: 294
Item Nbr:1:  scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-08-17/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=US/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-17.csv successfully copied
Item Nbr:2:  scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-08-03/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=US/tribe_wod-na-us-detailsalestrafficbychilditem-2021-08-03.csv successfully copied
Item Nbr:3:  scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-06-06/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=US/tribe_wod-na-us-detailsalestrafficbychilditem-2021-06-06.csv successfully copied
Item Nbr:4:  scrapers/seller_central/detail_page_sales_and_traffic_by_child_item_by_asin/p_creation_dt=2021-10-12/p_brand_id=TRIBE_WOD/p_region_id=NA/p_country_id=CA/tribe_wod-na-ca-detailsalestrafficbychilditem-2021-10-12.csv succes